Целта на задачата беше да подобрим резултата на лекции (-0.422).
Зареждаме сет-а

In [196]:
%matplotlib inline
import pandas as pd
train = pd.read_csv("/home/string/dev/machine-learning/supervised-learning/strain.csv", index_col="id")

In [197]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score

Успях да подобря резултата -> (-0.375). Мисля че е добро подобрение считайки колко малко трябваше да променя за да се стигне до него.
Премахнах от хиперпараметрите на Vectorizer-a max_df и min_df, което означава че са сложени по default на 1.0 и 1 и това даде доста голямо подобрение. ngram_range си се оказа оптимално, също така и lowercase=False. пробвах със други Naive Bayes модели, но MultinomialNB работеше по добре, и то със alpha=0.03, давайки малко подобрение

Накрая реших да погледна кои са другите хиперпараметри на vectorizer-a, тъй кате те направиха най големите подобрения. Единствено реших да пробвам token_pattern=’(?u)\b\w\w+\b’, какви думи взима като regex, и видях че не взима точно всичко от изречението. Затова за да генерализирам още подадох на token_pattern '\w+' да взима просто всичко което е дума. Това даде също добро подобрение и с това спрях да пробвам.

In [202]:
pipe = Pipeline([
    ('features', TfidfVectorizer(ngram_range=(1, 2), lowercase=False, sublinear_tf=True, token_pattern=r'\w+')),
    ('nb', MultinomialNB(alpha=0.03))
])

print(cross_val_score(pipe, train.text, train.author, cv=3, n_jobs=4))
print(cross_val_score(pipe, train.text, train.author, cv=3, n_jobs=4, scoring='neg_log_loss'))

[ 0.85523897  0.85366227  0.85318008]
[-0.3747073  -0.37280312 -0.3782487 ]


In [203]:
pipe.fit(train.text, train.author)
test = pd.read_csv("/home/string/dev/machine-learning/supervised-learning/stest.csv", index_col='id')
result = pipe.predict_proba(test.text.values)
pd.DataFrame(result, index=test.index, columns=authors).to_csv('result.csv')

Със този модел в kaggle получавам score от 0.33 и бях на 106 място. Може да не съм правил много графики или разсъждения в този notebook но задачата е изпълнена :)